### Goals

New Concepts:

* Using chat messages as our graph state. In the simple-graph notebook we used a string.
* Using chat models in graph nodes.
* Adding tools (aka Binding tools) to our chat model.
* Executing tool calls in graph nodes.

![Screenshot 2024-08-21 at 9.24.03 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbab08dd607b08df5e1101_chain1.png)

Some sample messages to give as input to chat models. 

Langchain contains AIMessage, HumanMessage, SystemMessage, ToolMessage as types of messages.


In [1]:
from langchain_core.messages import AIMessage, HumanMessage

messages = [AIMessage(content=f"How is your Learning going on so far?", name="Model")]
messages.append(HumanMessage(content=f"Decent",name="lol"))
messages.append(AIMessage(content=f"Great, what are you upto now?", name="Model"))
messages.append(HumanMessage(content=f"I want to learn about the tool calling", name="lol"))

## Chat Models

These models take messages as input and gives message as output.

I am trying to use local model that is available here, lets see how many errors i would face :)

In [5]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="qwen2.5:7b")
response = llm.invoke(messages)
type(response)


langchain_core.messages.ai.AIMessage

In [6]:
response

AIMessage(content='Sure! Tool calling refers to using tools or external programs from within a programming environment. This can be incredibly useful for leveraging specialized functions that aren\'t available directly in your primary programming language. Here’s an overview of how you might implement it:\n\n### 1. **Understanding the Basics**\n   - **Purpose:** To execute external commands or scripts.\n   - **Languages and Frameworks:** Commonly used languages include Python, Bash (for Unix-like systems), and PowerShell (for Windows).\n\n### 2. **Python Example**\nIn Python, you can use the `subprocess` module to call external tools.\n\n#### Using `subprocess.run()`\n```python\nimport subprocess\n\n# Run a command\nresult = subprocess.run([\'ls\', \'-l\'], capture_output=True, text=True)\nprint("Command output:", result.stdout)\n\n# Run another command with input\noutput = subprocess.run([\'grep\', \'filename.txt\'], input=b\'hello\\nworld\\nfoo\\nbar\', capture_output=True, text=True

In [7]:
response.response_metadata

{'model': 'qwen2.5:7b',
 'created_at': '2025-04-14T04:27:11.0446766Z',
 'done': True,
 'done_reason': 'stop',
 'total_duration': 97742225300,
 'load_duration': 38750000,
 'prompt_eval_count': 71,
 'prompt_eval_duration': 2605000000,
 'eval_count': 564,
 'eval_duration': 95090000000,
 'message': Message(role='assistant', content='', images=None, tool_calls=None)}

## Tools
Tools are for interacting with external systems(API's, Function calls, etc.).

They need a schema to be followed and not natural language.

The model will choose to call the tool building the schema for it appropriately.

We can pass functions as well, so let us pass two functions and see what happens. 

Multiply and subtract are the functions we shall try on.



In [8]:
def multiply(a:int, b:int)->int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a*b
def subtract(a:int, b:int)->int:
    """I want to see if LLM would understand and return correctly in the absence of explanation."""
    return a-b

llm_with_tools = llm.bind_tools([multiply, subtract])


In [11]:
tool_call = llm_with_tools.invoke([HumanMessage(content=f"What is 2 subtracted from 3", name="Lance")])

I might sound foolish for stating this, but i actually wanted to check if it would understand the sentence properly and assign 3 to `a` and 2 to `b` or simply assign the numbers in order. 

* Note that i have not provided any explanation for the subtract function. So i guess it understand the function schema for processing.

Good LLM.

In [ ]:
tool_call.tool_calls

[{'name': 'subtract',
  'args': {'a': 3, 'b': 2},
  'id': '15110266-7380-463f-9587-edd76a64e1a4',
  'type': 'tool_call'}]

### Building the state of the graph

As stated above we use messages for state here.

And the problem we face is that at every node the state is overwritten. But we want the messages to be appended at each node.

To address this [reducer functions](https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers) are introduced. A very good explanation is provided in the link.



In [14]:
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class MessageState(TypedDict):
    message_state: Annotated[list[AnyMessage], add_messages]


### Building Graph

A straight forward graph, with a START, TOOL_CALLING_LLM, END.



In [21]:
def tool_calling_llm(state: MessageState):
    return {"message_state": [llm_with_tools.invoke(state["message_state"])]}

In [22]:
from langgraph.graph import START, END, StateGraph

builder = StateGraph(MessageState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_edge(START, "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)

graph = builder.compile()


In [24]:
messages = graph.invoke({"message_state": HumanMessage(content = "subtract 2 from 3")})
messages['message_state']

[HumanMessage(content='subtract 2 from 3', additional_kwargs={}, response_metadata={}, id='879d9f1e-66ca-4ffa-89bb-3a8d4a65be68'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-04-14T05:59:30.3870301Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3060709200, 'load_duration': 43355700, 'prompt_eval_count': 239, 'prompt_eval_duration': 619000000, 'eval_count': 25, 'eval_duration': 2393000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-08607036-9dad-4f00-9cc7-2329001f5d56-0', tool_calls=[{'name': 'subtract', 'args': {'a': 3, 'b': 2}, 'id': 'e938ed81-263a-45cb-92cf-c4ec0ebd9d16', 'type': 'tool_call'}], usage_metadata={'input_tokens': 239, 'output_tokens': 25, 'total_tokens': 264})]

# Done with this notebook.